# Automated ML

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Udacity-exp-p3'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AZ52J4YUY to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Telecom Churn"
description_text = "Telecom Churn"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/nandex7/Capstone-Project---Azure-Machine-Learning-Engineer/main/data/WA_Fn-UseC_-Telco-Customer-Churn.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


In [5]:
dataset.take(5).to_pandas_dataframe()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,True,False,1,False,No phone service,DSL,No,...,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,5575-GNVDE,Male,0,False,False,34,True,No,DSL,Yes,...,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,3668-QPYBK,Male,0,False,False,2,True,No,DSL,Yes,...,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,7795-CFOCW,Male,0,False,False,45,False,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,9237-HQITU,Female,0,False,False,2,True,No,Fiber optic,No,...,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True


In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-automl"
# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded...................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

This creates a general AutoML settings object. These inputs must match what was used when training in the portal. label_column_name has to be y for example.

In [8]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5,
    "featurization": 'auto'
}
# TODO: Put your automl config here
automl_config = AutoMLConfig(task = "classification",
                             training_data=dataset,
                             compute_target=compute_target,
                             label_column_name="Churn",   
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:

from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)
assert(remote_run.get_status() == 'Completed')

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_run, fitted_model = remote_run.get_output()
best_metrics = best_run.get_metrics()
print(best_run)


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: Udacity-exp-p3,
Id: AutoML_225e7448-e7f7-40b2-a650-a845f51ff110_21,
Type: azureml.scriptrun,
Status: Completed)


In [15]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Udacity-exp-p3,AutoML_225e7448-e7f7-40b2-a650-a845f51ff110_21,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [16]:
fitted_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [17]:
best_metrics

{'average_precision_score_micro': 0.8942798556495799,
 'AUC_micro': 0.8929514138774506,
 'precision_score_micro': 0.8093163268597975,
 'f1_score_weighted': 0.8008333135826516,
 'accuracy': 0.8093163268597975,
 'average_precision_score_macro': 0.8038329659572003,
 'weighted_accuracy': 0.8668215440391828,
 'balanced_accuracy': 0.7192804416787397,
 'matthews_correlation': 0.4794080606696457,
 'log_loss': 0.42813699818168544,
 'f1_score_macro': 0.7348625464914171,
 'recall_score_micro': 0.8093163268597975,
 'average_precision_score_weighted': 0.8672027546118162,
 'recall_score_macro': 0.7192804416787397,
 'f1_score_micro': 0.8093163268597975,
 'AUC_macro': 0.8503873424624363,
 'recall_score_weighted': 0.8093163268597975,
 'AUC_weighted': 0.8503873424624363,
 'norm_macro_recall': 0.43856088335747945,
 'precision_score_weighted': 0.7999100195435476,
 'precision_score_macro': 0.762097108637492,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_225e7448-e7f7-40b2-a650-a845f51ff110

In [21]:
import joblib 

#best_run, fit_model = remote_run.get_output()

joblib.dump(value=fitted_model, filename="./BestAutoMLmodel.pkl")

['./output/BestAutoMLmodel.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [34]:
script = './score.py'
best_run.download_file('./outputs/scoring_file_v_1_0_0.py', './score.py')


In [37]:
model = remote_run.register_model(model_name= 'BestAutoMLmodel')

In [44]:
from azureml.core.model import InferenceConfig
from  azureml.core.environment import Environment
from azureml.core import Model
from azureml.core.webservice import AciWebservice

env = best_run.get_environment()    

inference_config = InferenceConfig(entry_script=script,
                                   environment=env)

deployment_config =AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1,
                                               enable_app_insights=True,
                                               auth_enabled=True,
                                              )

aservice = Model.deploy(ws, "telcomdeploy", [model], inference_config, deployment_config)
aservice.wait_for_deployment(show_output = True)
print(aservice.state)

print(aservice.scoring_uri)

pri_key, sec_key = aservice.get_keys()
print(pri_key)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://3e517a9d-f6cd-435d-a5a4-94482adc2cf5.southcentralus.azurecontainer.io/score
hZi5RJM9CQUAZbeynVztsPasEb392QOp


In [45]:
print(aservice.scoring_uri)

http://3e517a9d-f6cd-435d-a5a4-94482adc2cf5.southcentralus.azurecontainer.io/score


In [73]:
#Select all columns BUT not the target=CHURN .

test_df=df.iloc[:,:-1]
test_df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges'],
      dtype='object')

TODO: In the cell below, print the logs of the web service and delete the service

In [74]:
import requests
import json
scoring_uri='http://3e517a9d-f6cd-435d-a5a4-94482adc2cf5.southcentralus.azurecontainer.io/score'
primary,secondary = aservice.get_keys()
key =primary
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header

#test_sample = json.dumps(data)

input_data = json.dumps({
    'data': test_df[0:5].to_dict(orient= 'records')
})
print(input_data)

{"data": [{"customerID": "7590-VHVEG", "gender": "Female", "SeniorCitizen": 0, "Partner": true, "Dependents": false, "tenure": 1, "PhoneService": false, "MultipleLines": "No phone service", "InternetService": "DSL", "OnlineSecurity": "No", "OnlineBackup": "Yes", "DeviceProtection": "No", "TechSupport": "No", "StreamingTV": "No", "StreamingMovies": "No", "Contract": "Month-to-month", "PaperlessBilling": true, "PaymentMethod": "Electronic check", "MonthlyCharges": 29.85, "TotalCharges": 29.85}, {"customerID": "5575-GNVDE", "gender": "Male", "SeniorCitizen": 0, "Partner": false, "Dependents": false, "tenure": 34, "PhoneService": true, "MultipleLines": "No", "InternetService": "DSL", "OnlineSecurity": "Yes", "OnlineBackup": "No", "DeviceProtection": "Yes", "TechSupport": "No", "StreamingTV": "No", "StreamingMovies": "No", "Contract": "One year", "PaperlessBilling": false, "PaymentMethod": "Mailed check", "MonthlyCharges": 56.95, "TotalCharges": 1889.5}, {"customerID": "3668-QPYBK", "gender

In [75]:
headers['Authorization'] = f'Bearer {key}'
response = requests.post(scoring_uri, input_data, headers=headers)
output_data = response.text

print(output_data)


"{\"result\": [true, false, false, false, true]}"


In [77]:
data = {"data":
        [
          {
            "CustomerID":'7590-VHVEG',
            "gender": 68,
            "SeniorCitizen": 1,
            "Partner":True ,
            "Dependents": False,
            "tenure": 1,
            "PhoneService": True,
            "MultipleLines": 'No',
            "InternetService": 'DSL',
            "OnlineSecurity": 'No',
            "OnlineBackup": 'Yes',
            "DeviceProtection": 'No',
            "TechSupport": 'No',
            "StreamingTV": 'No',
            "StreamingMovies": 'No',
            "Contract":'Month-to-month',
            "PaperlessBilling":True,
            "PaymentMethod":'Electronic check',
            "MonthlyCharges":29.85,
            "TotalCharges":29.85
            
          }
      ]
    }

In [ ]:
aservice.delete()
print("Service deleted")

In [ ]:
compute_target.delete()
print("Compute deleted")